In [19]:
import os
import time
import importlib
import transformers
from datasets import load_dataset, load_from_disk
import torch
import re
import jinja2

import rpbuild as rp
import rpbuild.char
import rpbuild.data
import rpbuild.writer
import rpbuild.director
import rpbuild.roleplay

from rpbuild import load_template

# Trigger dynamic reload of module -- for editing without restarting the kernel
importlib.reload(rp.writer)

<module 'rpbuild.writer' from '/home/dinalt/rust/ai_development/roleplay_build/rpbuild/writer.py'>

### Load Resources
Load dataset and model for testing...

In [2]:
from transformers import BitsAndBytesConfig

# Where are models stored?
models_dir = "/home/dinalt/ai_assets/models"

# Configure a model to use.
# The name of this model -- which will live in models_dir
model_name = "fhai50032_RolePlayLake-7B" # AKA "fhai50032/RolePlayLake-7B"
model_id = os.path.join(models_dir, model_name)

# Or... or load it from the hub / cache
#model_id = "fhai50032/RolePlayLake-7B",

# The location of the input dataset
dataset_id = "dinalt/rpbuild"

# Device to run model on
device = 0

# Load dataset
dataset = load_dataset(dataset_id)["train"]
print(dataset)

# Load model with quantization
# Disable quantization, if you have enough memory. In this case, set device=0.
# See link for configuration options alternatives
# https://huggingface.co/docs/transformers/main/en/quantization

# Load model and tokenizer
causal_lm = rp.CausalLM(
    model_id,
    device=None,
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
    #device_map="auto",
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
    )
)

Resolving data files:   0%|          | 0/538 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['preset', 'proxy', 'summary', 'example_dialog', 'director_log', 'plist', 'description', 'char_name', 'pairing_reason', 'scenario', 'greeting', 'conversation'],
    num_rows: 4309
})
Tokenizer uses "right" padding; this may require moving it to "left" for batch generation.


`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=2)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralFlashAttention2(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNor

### Examine Characters
Get a random character from the dataset and dump it

In [3]:
#transformers.set_seed(42)
#example_char = rp.data.random_char(dataset)
example_char = dataset[0]
rp.data.dump_character_data(example_char)


char_name: Kenji Suzuki

summary: Kenji Suzuki's talent lies in his ability to adapt and thrive in diverse environments. Whether navigating treacherous terrain or facing unexpected challenges, his adaptability and resilience make him a valuable asset in the unpredictable post-apocalyptic world.

preset: Midnight-Enigma

pairing_reason: The contrast between Kenji Suzuki's adaptability in diverse environments and Talon Shadowclaw's stealth and deception skills creates an interesting dynamic for collaboration and resolving difficult situations together.
==================================description===================================

Name: Kenji Suzuki

Human
Age: Early 30s

Feature: Short black hair, piercing brown eyes, athletic build

Personality: resourceful, adaptable, quick-thinking, resilient, adventurous

Loves: Exploring new places, learning survival skills, helping others

Description:
Kenji Suzuki was born and raised in Japan but his thirst for adventure led him to travel the 

## Create Test Character

Get a random character (meta-data) and a random character to represent the user.

Instantiate a "Character" object from the meta-data.

In [8]:
transformers.set_seed(44)
example_char = rp.char.CharMeta.from_data(rp.data.random_char(dataset))
example_user = rp.char.CharMeta.from_data(rp.data.random_char(dataset))
rp.data.print_char_summary(example_char, example_user)

char = rp.char.Character(
    char_meta=example_char,
    causal_lm=causal_lm,
    generation_config="Midnight-Enigma",
    user_meta=example_user,
)

Token indices sequence length is longer than the specified maximum sequence length for this model (831 > 255). Running this sequence through the model will result in indexing errors


Char: Nana Hoshimi

Char Summary: A shy and introverted girl who discovers her latent psychic powers after a near-death experience. Nana's ability to see the future allows her to aid the Supernatural Detective Agency in preventing disasters and thwarting evil plots.

Proxy User: Ibn Battuta Noor

Proxy User Summary: A renowned Moroccan scholar and explorer, Ibn Battuta Noor journeys across vast deserts, traverses towering mountains, and sails uncharted waters, documenting his adventures in a captivating travelogue. His insatiable thirst for knowledge makes him a pioneer in the field of exploration.



### Show Character's System Prompt
Dump out the characters system prompt.

In [9]:
print(char.system_prompt())

Name: Nana Hoshimi

Human, early twenties

Soft spoken with a timid demeanor, Nana Hoshimi is a young woman who prefers the company of books over crowds. Her long, dark hair cascades down her shoulders in loose waves, framing her delicate features. Her large, brown eyes seem to carry the weight of the world, revealing the depth of her introspective nature.

Nana was an unassuming, quiet student until a near-death experience awakened her latent psychic abilities. Since then, she's discovered she can see glimpses of the future. Shocked and unsure of how to control these newfound powers, she sought help from the Supernatural Detective Agency.

Despite her initial reluctance to get involved, Nana soon realized that she could use her gift to prevent disasters and thwart evil plots. Though still nervous and easily startled, she finds solace in knowing that she can make a difference. With each successful intervention, her confidence grows.

In her spare time, Nana enjoys curling up with a goo

### Add Character's Greeting
Add character's greeting to the conversation, then print the conversation.

In [10]:
char.greet()
char.print_conversation()

--------------------------------- system (831) ---------------------------------
Name: Nana Hoshimi

Human, early twenties

Soft spoken with a timid demeanor, Nana Hoshimi is a young woman who prefers the company of books over crowds. Her long, dark hair cascades down her shoulders in loose waves, framing her delicate features. Her large, brown eyes seem to carry the weight of the world, revealing the depth of her introspective nature.

Nana was an unassuming, quiet student until a near-death experience awakened her latent psychic abilities. Since then, she's discovered she can see glimpses of the future. Shocked and unsure of how to control these newfound powers, she sought help from the Supernatural Detective Agency.

Despite her initial reluctance to get involved, Nana soon realized that she could use her gift to prevent disasters and thwart evil plots. Though still nervous and easily startled, she finds solace in knowing that she can make a difference. With each successful interven

### Insert User Dialog
Add user dialog to the history.

In [11]:
char.user_says("I'd love to learn about ancient Egypt.")

{'role': 'user',
 'content': "I'd love to learn about ancient Egypt.",
 'tokens': 11,
 'name': 'Ibn Battuta Noor'}

### Show Character's Chat Prompt

In [12]:
print(char.chat_prompt())

Name: Nana Hoshimi

Human, early twenties

Soft spoken with a timid demeanor, Nana Hoshimi is a young woman who prefers the company of books over crowds. Her long, dark hair cascades down her shoulders in loose waves, framing her delicate features. Her large, brown eyes seem to carry the weight of the world, revealing the depth of her introspective nature.

Nana was an unassuming, quiet student until a near-death experience awakened her latent psychic abilities. Since then, she's discovered she can see glimpses of the future. Shocked and unsure of how to control these newfound powers, she sought help from the Supernatural Detective Agency.

Despite her initial reluctance to get involved, Nana soon realized that she could use her gift to prevent disasters and thwart evil plots. Though still nervous and easily startled, she finds solace in knowing that she can make a difference. With each successful intervention, her confidence grows.

In her spare time, Nana enjoys curling up with a goo

### Generate a Response
Generate a response to the history.

In [13]:
transformers.set_seed(32)
outputs = char.generate(auto_add=True)
print(outputs["response"])

*excitedly* Oh, Egyptian history is so fascinating! There's something incredibly mystical about the civilization that left such an indelible mark on humanity. If you want to dive deeper into their culture and mythology, allow me to recommend a few titles that will surely pique your interest.


## Test Full Pipeline

### Choose Test Characters
Select a random character from the dataset as the primary character.  
Then make a list of N random proxy-user characters and ask the Writer to identify which of these N would make for the best story.

In [20]:
transformers.set_seed(16)
example_char = rp.char.CharMeta.from_data(rp.data.random_char(dataset))

# Ask the write to pair character with the best match.
writer = rp.writer.Writer(causal_lm, debug_level=2)
result = writer.choose_supporting_character(example_char, rp.writer.get_random_proxy_users(dataset, example_char, n=5))

print(f"{'':-^80}")
print(f"Writer selected {result['name']} because: {result['reason']}")
print(f"{'':-^80}")

example_user = result["meta"]
rp.data.print_char_summary(example_char, example_user)

# Get random generation presets for each character
char_preset = rp.model.random_preset()
user_preset = rp.model.random_preset()
print(f"Character Preset: {char_preset}")
print(f"Character Preset: {user_preset}")

----------------------------- Writer Prompt (1807) -----------------------------
### Instruction:
The following character will be the main character in a story.

Main Characer: Sophia Patel
A young and dynamic politician known for her tech-savvy approach to governance, Sophia Patel is transforming her nation's digital infrastructure and promoting innovation across all sectors.
[Sophia Patel: human, dark brown skin, almond-shaped eyes with golden flecks, wavy black hair, prominent cheekbones, warm smile, early 30s, politician, Minister of Digital Transformation and Innovation, degrees in Computer Science and Political Science, speaks fluently in English, Mandarin, and Spanish, driven, charismatic, passionate about technology and progress, values transparency, fairness, collaboration; Loves: brainstorming innovative solutions, debating policy issues, collaborating with experts, traveling, learning about cultures, spending time with family and friends; Professional attire when in office, 

### Write Plot Outline
Have the write write a script for the pair of characters.

In [21]:
transformers.set_seed(43)

script = writer(example_char, example_user)
print(f"{'script':-^80}")
print(script)

----------------------------- Writer Prompt (722) ------------------------------
### Instruction:
[Aurora Lunawood: elven entity infused with nature's essence; Skin: ethereal glowing alabaster complexion; Eyes: vibrant emerald green (wisdom, compassion); Hair: long, wavy chestnut locks (shimmers with sunlight); Build: tall and slender (around 6 feet); Age: over 300 years old; Home: Eldertree Grove (mystical forest, lush foliage, crystal clear waters, blooming flowers); Persona: serene and patient, embodies healing, gentle and kind, values harmony, attentive listener, empathetic companion; Abilities: restores damaged ecosystems; Spare Time: meditates, communes with elements, shares stories and wisdom; Clothes: flowing robes (natural fibers, leaves, vines, feathers); Environment: Eldertree Grove (nestled deep within, surrounded by lush foliage, crystal clear waters, blooming flowers)]

[Sophia Patel: human, dark brown skin, almond-shaped eyes with golden flecks, wavy black hair, prominen

### Automation Infrastructure

We create two character instances from the character meta-data selected by the "Writer."

The first of these, "char," represents the AI character. The second, "proxy_user," is a stand-in AI character for a real user. They each have their own context and history. The dialog is shared, but the character specific elements of the contexts differ. The primary character will use the "canned" greeting message as the first generation, while the proxy user generates a response to the greeting.

We create a "Director," which is responsible for providing guidance to the chracters. The director uses the plot outline, generated by the "Writer," to provide something resembling direction to the interaction. The director has access to the scenario, while the characters do not. The director's role is roughly analogeous to a GM in a role-play session. There is a general scenario plan, but most of the details are left up to the characters. Much is the case with a real roleplay session, the characters can potentially deviate from the prepared scenario.

The "Roleplay" object drives the interaction among the two AI characters and the director. Turns are taken generating dialog between the two characters.

The Roleplay can be advanded a single step at a time for diagnostics or allowed to run until a target number of tokens have been generated or the plot reaches the conclusion.

There is an optional "dialog_filter" object which preprocesses the generated outputs and can control the generation flow. This typically involves some combination of regex and query prompts to evaluate conditions -- for example, trigger a "retake," if a bad generation is identified.

#### Character Prompting
The prompt shown to the character for each generation is not simply a system message, followed by exchanges between "user" and "assistant." Instead, we create a single unified instruction prompt, without an explicit system message, from the character, director, and dialog state. The format is roughly like this:

- begin instruction token
- character description
- dialog examples (limited to one, for context space)
- begin roleplya token
- truncated dialog history
- character's plist
- static dialog instruction
- director's next instruction
- end instruction token
- begin response token
- "{{char}}:"

The dialog history is truncated via a configurrable token limit, so the character does not always have full-lookback.

We put a reminder of who they are playing, the plist, near the end. This helps them stay in character.

Some general instructions are added, "Write the next response for {{char}}...," with dialog formatting guidelines and examples.

Finally, the most recent output from the director is concatenated. Each characer only sees their own director messages and they only see the most recent director instruction.

As such, the directors ouput looks like part of the instructions. Functionally, it acts somewhat like internal thoughts and rehersal for the character, providing a chain-of-thought like reasoning step and working memory, prior to generation. Looking at the actual generations, they seem to function very much like our concept of excecutive function.

Note that each character has limited direct information about their counterpart. On the first proxy user cycle, the director instructs the character to introduce themselves, but otherwise, they know little directly about each other.

The director has access to both character descriptions, so can help fill this void indirectly.

It may be worth experimenting with truncating the character's dialog history down to a minimum, instead relying on the Director for remembering the history and what they should be doing. This may help the characters focus on their immediate task -- generating good dialog for the next response. This would have the added advantage of keeping the first part of the prompt closer to the next generation.

#### Director Prompting
- begin instruction token
- general instructions to act as a director
- user-plist
- char-plist
- scenario
- truncated dialog history
- outline of responsibilities
- instruciton to instruct char
- Conditional instruction to instruct the proxy user to introduce themselves after the character greeting.
- end instruction token
- being response token
- "Director:"

Unlike the characters, the director has access to both character descriptions, the scenario created by the writer, and a longer (but still truncated) dialog between the characters.

While we retain the directory dialog history, it is not present in either the director's or characters' prompts.

In [23]:
transformers.set_seed(42)

#dialog_filter = DialogFilter(causal_lm, debug=True)

dialog_filter=None
char = rp.char.Character(
    char_meta=example_char,
    causal_lm=causal_lm,
    generation_config=char_preset,
    user_meta=example_user,
    gen_post_process=dialog_filter,
    # Shows character prompts and internal diagnostics
    debug=False,
    history_token_limit=1800
)

proxy_user = rp.char.Character(
    char_meta=example_user,
    causal_lm=causal_lm,
    generation_config=user_preset,
    user_meta=example_char,
    gen_post_process=dialog_filter,
    debug=False,
    history_token_limit=1800
)

director = rp.director.Director(
    causal_lm,
    script=script,
    # Shows director prompts and more...
    debug=False,
    history_token_limit=2000
)

roleplay = rp.roleplay.Roleplay(
    char=char,
    user=proxy_user,
    scenario=script,
    director=director,

    # Shows generated dialog and control events
    debug=True
)

### Generate Next Dialog Step
Run to perform a single dialog generation step.

The first call will user the "canned" greeting message. Actual generation will occur on subsequent calls.

In [24]:
output = roleplay.generate_next()

------------------------------ Sophia Patel (243) ------------------------------
Aurora Lunawood, it's a pleasure to make your acquaintance once more! As you enter the conference hall, Sophia Patel, Minister of Digital Transformation and Innovation, rises from her seat and extends a hand towards you with a warm smile. Dressed impeccably in her power suit, her dark brown eyes gleam with intelligence and enthusiasm.

Sophia: Hello there! It seems we've crossed paths again. I was just reflecting on how fortunate it is that our paths keep intersecting – the world could always benefit from another insightful discussion. Take a seat beside me, won't you?

As you settle into the chair next to her, Sophia continues,

Sophia: I've been giving quite a bit of thought lately to groundbreaking technological policies and innovations that could shape our collective future. Fancy joining me for a stimulating exchange of ideas? Perhaps we could delve into the potential of artificial intelligence or dis

### Generate Target Tokens
Generate until the target number of tokens has been reached.

In [25]:
start = time.perf_counter()
conversations = roleplay(2000)
end = time.perf_counter()
print(f"Elapsed {(end-start):.03f} secs.")

-------------------------------- Director (395) --------------------------------
Aurora Lunawood, as you settle into the chair next to Sophia Patel, gently close your eyes for a moment and connect with the energy around you. After a beat, gradually open your eyes and begin describing yourself, drawing upon your unique connection with nature and your role as an empathetic listener and collaborator.

Aurora Lunawood: Hello Sophia, I am Aurora Lunawood. My essence is intertwined with nature itself, giving me an ethereal glow and allowing me to restore damaged ecosystems. I embody healing energies and cherish harmony above all else. I have lived for centuries in the mystical Eldertree Grove, surrounded by lush vegetation, crystal-clear waters, and blooming flowers. My time there has taught me the beauty of listening, understanding different perspectives, and finding common ground in empathy. Today, I am eager to share my insights with you and combine our knowledge to create a future where 

#### Get Conversation Token Count

In [26]:
print(char.count_tokens())

2011


#### Dump Generated Conversation

In [27]:
#print_conversation(char.conversation, char.director_log)
char.print_conversation()

--------------------------------- system (852) ---------------------------------
Name: Sophia Patel

Human
Dark brown skin, almond-shaped eyes with flecks of gold, wavy black hair, framed by prominent cheekbones and a warm smile
Early 30s

Politician, serving as Minister of Digital Transformation and Innovation for her nation
Degrees in Computer Science and Political Science from prestigious universities
Speaks fluently in multiple languages including English, Mandarin, and Spanish

Sophia is driven, charismatic, and passionate about leveraging technology to improve lives and promote progress. Her natural ability to connect with people and articulate complex ideas in simple terms has earned her a dedicated following. She values transparency, fairness, and collaboration, often bringing diverse groups together to tackle challenges.

Loves: Brainstorming innovative solutions, debating policy issues, collaborating with experts from various fields, traveling to learn about different culture